In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import *

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
data_csvfile = pd.read_csv('/content/lab4_ex3.csv')

In [ ]:
print(type(data_csvfile))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
data_csvfile.columns

Index(['Drone Model', 'Carrying Capacity', 'Cost', 'Parking space'], dtype='object')

In [ ]:
data_csvfile['Drone Model']

0     MANAS
1       JAY
2      VEER
3    SPASHT
4    DHAKSH
Name: Drone Model, dtype: object

In [ ]:
data_csvfile['Carrying Capacity']

0    12
1    14
2    10
3    15
4    14
Name: Carrying Capacity, dtype: int64

In [ ]:
data_csvfile['Cost']

0     6.0
1     7.5
2     9.5
3    10.0
4    12.5
Name: Cost, dtype: float64

In [ ]:
data_csvfile['Parking space']

0    4.25
1    3.25
2    2.50
3    3.45
4    2.25
Name: Parking space, dtype: float64

In [ ]:
data_csvfile.index

RangeIndex(start=0, stop=5, step=1)

##Ans 4 :

Decision Variables : $x_i=$ Number of drones of model i, $x_i\in\mathbb{Z^{+}}, i = 1,2,3,4,5$

i = 1 = Manas

i = 2 = Jay

i = 3 = Veer

i = 4 = Spasht

i = 5 = Dhaksh

$\text{max  } 12x_1+14x_2+10x_3+15x_4+14x_5$

subject to:

$\bullet 6x_1+7.5x_2+9.5x_3+10x_4+12.5x_5\leq59\\
\bullet 4.25x_1+3.25x_2+2.5x_3+3.45x_4+2.25x_5\leq73\\
\bullet x_i\in\mathbb{Z^{+}}, i = 1,2,3,4,5$

In [ ]:
model = ConcreteModel()

In [ ]:
N = len(data_csvfile.index)

In [ ]:
M = 2

In [ ]:
col_indices = range(N)

In [ ]:
model.x = Var(col_indices,domain = NonNegativeIntegers)

In [ ]:
model.objective = Objective(expr = summation(data_csvfile['Carrying Capacity'], model.x),sense=maximize)

In [ ]:
model.constraints = ConstraintList()

In [ ]:
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 59 )

In [ ]:
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 73 )

In [ ]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  114.0

Decision Variables
x[ 0 ] =  6.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  58.5 :  59.0
      2 :  None : 35.25 :  73.0


##Ans 6 :

The objective function value at optimality is 114.

$x_1$, quantity of MANAS model = 6

$x_2$, quantity of JAY model = 3

$x_3$, quantity of VEER model = 0

$x_4$, quantity of SPASHT model = 0

$x_5$, quantity of DHAKSH model = 0

The constraint 1 = 58.5 and is not satisfied to equality.

The constraint 2 = 35.25 and is not satisfied to equality.



In [ ]:
model_y = ConcreteModel()
model_y.y = Var(col_indices)

In [ ]:
model_y.objective_y = Objective(expr = summation(data_csvfile['Carrying Capacity'], model_y.y),sense=maximize)

In [ ]:
model_y.constraints_y = ConstraintList()
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 59 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 73 )

In [ ]:
lower_bound = 0
upper_bound = np.inf
for i in col_indices:
  model_y.y[i].setlb(lower_bound)
  model_y.y[i].setub(upper_bound)

In [ ]:
model_y.pprint()

2 Set Declarations
    constraints_y_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    y : Size=5, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective_y : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*y[0] + 14*y[1] + 10*y[2] + 15*y[3] + 14*y[4]

1 Constraint Declarations
    constraints_y : Size=2

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  117.9999996

Decision Variables
x[ 0 ] =  9.8333333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=2
    Key : Lower : Body         : Upper
      1 :  None :   58.9999998 :  59.0
      2 :  None : 41.791666525 :  73.0


##Ans 7 :

The objective function value at optimality is 117.9999996.

$x_1=9.8333333$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 58.9999998 and is not satisfied to equality.

The constraint 2 = 41.791666525 and is not satisfied to equality.



##Ans 8 :

The solution of the MILP cannot be obtained by just rounding off the solution of the LP.

This is not possible since in the case of LP, it does not consider the fact that the quantity cannot be a decimal value.

In [ ]:
model_y.constraints_y[1].deactivate()
model_y.constraints_y[2].deactivate()

In [ ]:
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 71 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 87 )

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  141.999996

Decision Variables
x[ 0 ] =  11.833333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=4
    Key : Lower : Body        : Upper
      3 :  None :   70.999998 :  71.0
      4 :  None : 50.29166525 :  87.0


In [ ]:
model.constraints[1].deactivate()
model.constraints[2].deactivate()
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 71 )
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 87 )

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  138.0

Decision Variables
x[ 0 ] =  8.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=4
    Key : Lower : Body  : Upper
      3 :  None :  70.5 :  71.0
      4 :  None : 43.75 :  87.0


##Ans 9 :

###When changes done in LP :

The objective function value changes to 141.999996.

$x_1=11.833333$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 70.999998 and is not satisfied to equality.

The constraint 2 = 50.29166525 and is not satisfied to equality.

###When changes done in MILP :

The objective function value changes to 138.

$x_1$, quantity of MANAS model = 8

$x_2$, quantity of JAY model = 3

$x_3$, quantity of VEER model = 0

$x_4$, quantity of SPASHT model = 0

$x_5$, quantity of DHAKSH model = 0

The constraint 1 = 70.5 and is not satisfied to equality.

The constraint 2 = 43.75 and is not satisfied to equality.



In [ ]:
model_y.constraints_y[3].deactivate()
model_y.constraints_y[4].deactivate()
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 67 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 88 )

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  134.000004

Decision Variables
x[ 0 ] =  11.166667
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=6
    Key : Lower : Body        : Upper
      5 :  None :   67.000002 :  67.0
      6 :  None : 47.45833475 :  88.0


In [ ]:
model_y.constraints_y[5].deactivate()
model_y.constraints_y[6].deactivate()
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 65 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 89 )

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  129.999996

Decision Variables
x[ 0 ] =  10.833333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=8
    Key : Lower : Body        : Upper
      7 :  None :   64.999998 :  65.0
      8 :  None : 46.04166525 :  89.0


In [ ]:
model_y.constraints_y[7].deactivate()
model_y.constraints_y[8].deactivate()
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 63 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 90 )

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  126.0

Decision Variables
x[ 0 ] =  10.5
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=10
    Key : Lower : Body   : Upper
      9 :  None :   63.0 :  63.0
     10 :  None : 44.625 :  90.0


In [ ]:
model_y.constraints_y[9].deactivate()
model_y.constraints_y[10].deactivate()
model_y.constraints_y.add(expr = summation(data_csvfile['Cost'],model_y.y) <= 61 )
model_y.constraints_y.add(expr = summation(data_csvfile['Parking space'],model_y.y) <= 91 )

In [ ]:
result_y = opt_cbc.solve(model_y)
print('Solver status:', result_y.solver.status)
print('Solver termination condition:',result_y.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model_y.objective_y())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_y.y[i].value)

print('\nConstraints')
model_y.constraints_y.display()

Objective =  122.000004

Decision Variables
x[ 0 ] =  10.166667
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints_y : Size=12
    Key : Lower : Body        : Upper
     11 :  None :   61.000002 :  61.0
     12 :  None : 43.20833475 :  91.0


In [ ]:
model.constraints[3].deactivate()
model.constraints[4].deactivate()
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 67 )
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 88 )

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  132.0

Decision Variables
x[ 0 ] =  11.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=6
    Key : Lower : Body  : Upper
      5 :  None :  66.0 :  67.0
      6 :  None : 46.75 :  88.0


In [ ]:
model.constraints[5].deactivate()
model.constraints[6].deactivate()
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 65 )
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 89 )

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  126.0

Decision Variables
x[ 0 ] =  7.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=8
    Key : Lower : Body : Upper
      7 :  None : 64.5 :  65.0
      8 :  None : 39.5 :  89.0


In [ ]:
model.constraints[7].deactivate()
model.constraints[8].deactivate()
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 63 )
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 90 )

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  124.0

Decision Variables
x[ 0 ] =  8.0
x[ 1 ] =  2.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=10
    Key : Lower : Body : Upper
      9 :  None : 63.0 :  63.0
     10 :  None : 40.5 :  90.0


In [ ]:
model.constraints[9].deactivate()
model.constraints[10].deactivate()
model.constraints.add(expr = summation(data_csvfile['Cost'],model.x) <= 61 )
model.constraints.add(expr = summation(data_csvfile['Parking space'],model.x) <= 91 )

In [ ]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print('Objective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Objective =  120.0

Decision Variables
x[ 0 ] =  10.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=12
    Key : Lower : Body : Upper
     11 :  None : 60.0 :  61.0
     12 :  None : 42.5 :  91.0


##Ans 10 :

#In case of LP:

**When budget decreased to 67, and parking space increased to 88** 

The objective function value is : 134.000004.

$x_1=11.166667$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 67.000002 and is not satisfied to equality.

The constraint 2 = 47.45833475 and is not satisfied to equality.

**When budget decreased to 65, and parking space increased to 89** 

The objective function value is : 129.999996.

$x_1=10.833333$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 64.999998 and is not satisfied to equality.

The constraint 2 = 46.04166525 and is not satisfied to equality.

**When budget decreased to 63, and parking space increased to 90** 

The objective function value is : 126.0.

$x_1=10.5$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 63.0 and is satisfied to equality.

The constraint 2 = 44.625 and is not satisfied to equality.

**When budget decreased to 61, and parking space increased to 91** 

The objective function value is : 122.000004.

$x_1=10.166667$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 61.000002 and is not satisfied to equality.

The constraint 2 = 43.20833475 and is not satisfied to equality.


#In case of MILP :

**When budget decreased to 67, and parking space increased to 88** 

The objective function value is : 132.

$x_1=11$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 66 and is not satisfied to equality.

The constraint 2 = 46.75 and is not satisfied to equality.

**When budget decreased to 65, and parking space increased to 89** 

The objective function value is : 126.

$x_1=7$

$x_2=3$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 64.5 and is not satisfied to equality.

The constraint 2 = 39.5 and is not satisfied to equality.

**When budget decreased to 63, and parking space increased to 90** 

The objective function value is : 124.

$x_1=8$

$x_2=2$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 63 and is not satisfied to equality.

The constraint 2 = 40.5 and is not satisfied to equality.

**When budget decreased to 61, and parking space increased to 91** 

The objective function value is : 120.

$x_1=10$

$x_2=0$

$x_3=0$

$x_4=0$

$x_5=0$

The constraint 1 = 60 and is not satisfied to equality.

The constraint 2 = 42.5 and is not satisfied to equality.

$\bullet$ *The objective function value of the LP in one of the cases is more by around 2 than the objective function value of the MILP in that case.*

$\bullet$ *The value of the decision variables in the cases when budget is 67 and 61 with parking space 88 and 91 respectively can be rounded off in the LP to obtain the value of the decision variables in the case of MILP.*